# Fie Upon Thee, Autocorrect!

<img src="img/shakespeare.jpg" width="200">

<br><br><br>

## Recap of yesterday

<br><br><br>

## Dataset: the complete works of Shakespeare

In [1]:
import numpy as np

In [2]:
with open("data/shakespeare.txt") as file:
    corpus = file.read()

In [3]:
len(corpus)

5367504

In [4]:
print(corpus[100000:101000])

ook up that fire,
Which many legions of true hearts had warmed,
And so the general of hot desire,
Was sleeping by a virgin hand disarmed.
This brand she quenched in a cool well by,
Which from Love’s fire took heat perpetual,
Growing a bath and healthful remedy,
For men diseased; but I, my mistress’ thrall,
  Came there for cure and this by that I prove,
  Love’s fire heats water, water cools not love.

THE END

ALL’S WELL THAT ENDS WELL

Contents

ACT I
Scene I. Rossillon. A room in the Countess’s palace.
Scene II. Paris. A room in the King’s palace.
Scene III. Rossillon. A Room in the Palace.

ACT II
Scene I. Paris. A room in the King’s palace.
Scene II. Rossillon. A room in the Countess’s palace.
Scene III. Paris. The King’s palace.
Scene IV. Paris. The King’s palace.
Scene V. Another room in the same.

ACT III
Scene I. Florence. A room in the Duke’s palace.
Scene II. Rossillon. A room in the Countess’s palace.
Scene III. Florence. Before the Duke’s palace.
Scene IV. Rossillon. A roo

In [5]:
unique_characters = np.unique(np.frombuffer(corpus.encode(), np.uint8).reshape(-1, 1))
unique_characters

array([ 10,  32,  33,  35,  36,  37,  38,  40,  41,  42,  44,  45,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
        63,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  93,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
       107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
       120, 121, 122, 128, 132, 134, 135, 137, 147, 148, 152, 153, 156,
       157, 160, 162, 166, 167, 168, 169, 170, 171, 174, 187, 191, 195,
       197, 226, 239], dtype=uint8)

In [6]:
unique_characters.view("S1")

array([b'\n', b' ', b'!', b'#', b'$', b'%', b'&', b'(', b')', b'*', b',',
       b'-', b'.', b'/', b'0', b'1', b'2', b'3', b'4', b'5', b'6', b'7',
       b'8', b'9', b':', b';', b'?', b'A', b'B', b'C', b'D', b'E', b'F',
       b'G', b'H', b'I', b'J', b'K', b'L', b'M', b'N', b'O', b'P', b'Q',
       b'R', b'S', b'T', b'U', b'V', b'W', b'X', b'Y', b'Z', b'[', b']',
       b'a', b'b', b'c', b'd', b'e', b'f', b'g', b'h', b'i', b'j', b'k',
       b'l', b'm', b'n', b'o', b'p', b'q', b'r', b's', b't', b'u', b'v',
       b'w', b'x', b'y', b'z', b'\x80', b'\x84', b'\x86', b'\x87',
       b'\x89', b'\x93', b'\x94', b'\x98', b'\x99', b'\x9c', b'\x9d',
       b'\xa0', b'\xa2', b'\xa6', b'\xa7', b'\xa8', b'\xa9', b'\xaa',
       b'\xab', b'\xae', b'\xbb', b'\xbf', b'\xc3', b'\xc5', b'\xe2',
       b'\xef'], dtype='|S1')

In [7]:
np.count_nonzero(unique_characters <= ord("z"))

81

<br><br><br>

## In progress...

In [8]:
import sklearn.neural_network

In [164]:
text = """
WOULD YOU LIKE GREEN EGGS AND HAM?

I DO NOT LIKE THEM, SAM-I-AM.
I DO NOT LIKE GREEN EGGS AND HAM.

WOULD YOU LIKE THEM HERE OR THERE?

I WOULD NOT LIKE THEM HERE OR THERE.
I WOULD NOT LIKE THEM ANYWHERE.
I DO NOT LIKE GREEN EGGS AND HAM.
I DO NOT LIKE THEM, SAM-I-AM.

WOULD YOU LIKE THEM IN A HOUSE?
WOULD YOU LIKE THEN WITH A MOUSE?

I DO NOT LIKE THEM IN A HOUSE.
I DO NOT LIKE THEM WITH A MOUSE.
I DO NOT LIKE THEM HERE OR THERE.
I DO NOT LIKE THEM ANYWHERE.
I DO NOT LIKE GREEN EGGS AND HAM.
I DO NOT LIKE THEM, SAM-I-AM.

WOULD YOU EAT THEM IN A BOX?
WOULD YOU EAT THEM WITH A FOX?

NOT IN A BOX. NOT WITH A FOX.
NOT IN A HOUSE. NOT WITH A MOUSE.
I WOULD NOT EAT THEM HERE OR THERE.
I WOULD NOT EAT THEM ANYWHERE.
I WOULD NOT EAT GREEN EGGS AND HAM.
I DO NOT LIKE THEM, SAM-I-AM.
"""

words = np.array(text.replace(",", " ").replace(".", " ").replace("?", " ").replace("!", " ").replace("-", " ").split())
dictionary, text_as_integers = np.unique(words, return_inverse=True)

In [165]:
dictionary

array(['A', 'AM', 'AND', 'ANYWHERE', 'BOX', 'DO', 'EAT', 'EGGS', 'FOX',
       'GREEN', 'HAM', 'HERE', 'HOUSE', 'I', 'IN', 'LIKE', 'MOUSE', 'NOT',
       'OR', 'SAM', 'THEM', 'THEN', 'THERE', 'WITH', 'WOULD', 'YOU'],
      dtype='<U8')

In [166]:
len(dictionary)

26

In [167]:
text_as_integers

array([24, 25, 15,  9,  7,  2, 10, 13,  5, 17, 15, 20, 19, 13,  1, 13,  5,
       17, 15,  9,  7,  2, 10, 24, 25, 15, 20, 11, 18, 22, 13, 24, 17, 15,
       20, 11, 18, 22, 13, 24, 17, 15, 20,  3, 13,  5, 17, 15,  9,  7,  2,
       10, 13,  5, 17, 15, 20, 19, 13,  1, 24, 25, 15, 20, 14,  0, 12, 24,
       25, 15, 21, 23,  0, 16, 13,  5, 17, 15, 20, 14,  0, 12, 13,  5, 17,
       15, 20, 23,  0, 16, 13,  5, 17, 15, 20, 11, 18, 22, 13,  5, 17, 15,
       20,  3, 13,  5, 17, 15,  9,  7,  2, 10, 13,  5, 17, 15, 20, 19, 13,
        1, 24, 25,  6, 20, 14,  0,  4, 24, 25,  6, 20, 23,  0,  8, 17, 14,
        0,  4, 17, 23,  0,  8, 17, 14,  0, 12, 17, 23,  0, 16, 13, 24, 17,
        6, 20, 11, 18, 22, 13, 24, 17,  6, 20,  3, 13, 24, 17,  6,  9,  7,
        2, 10, 13,  5, 17, 15, 20, 19, 13,  1])

In [168]:
dictionary[text_as_integers]

array(['WOULD', 'YOU', 'LIKE', 'GREEN', 'EGGS', 'AND', 'HAM', 'I', 'DO',
       'NOT', 'LIKE', 'THEM', 'SAM', 'I', 'AM', 'I', 'DO', 'NOT', 'LIKE',
       'GREEN', 'EGGS', 'AND', 'HAM', 'WOULD', 'YOU', 'LIKE', 'THEM',
       'HERE', 'OR', 'THERE', 'I', 'WOULD', 'NOT', 'LIKE', 'THEM', 'HERE',
       'OR', 'THERE', 'I', 'WOULD', 'NOT', 'LIKE', 'THEM', 'ANYWHERE',
       'I', 'DO', 'NOT', 'LIKE', 'GREEN', 'EGGS', 'AND', 'HAM', 'I', 'DO',
       'NOT', 'LIKE', 'THEM', 'SAM', 'I', 'AM', 'WOULD', 'YOU', 'LIKE',
       'THEM', 'IN', 'A', 'HOUSE', 'WOULD', 'YOU', 'LIKE', 'THEN', 'WITH',
       'A', 'MOUSE', 'I', 'DO', 'NOT', 'LIKE', 'THEM', 'IN', 'A', 'HOUSE',
       'I', 'DO', 'NOT', 'LIKE', 'THEM', 'WITH', 'A', 'MOUSE', 'I', 'DO',
       'NOT', 'LIKE', 'THEM', 'HERE', 'OR', 'THERE', 'I', 'DO', 'NOT',
       'LIKE', 'THEM', 'ANYWHERE', 'I', 'DO', 'NOT', 'LIKE', 'GREEN',
       'EGGS', 'AND', 'HAM', 'I', 'DO', 'NOT', 'LIKE', 'THEM', 'SAM', 'I',
       'AM', 'WOULD', 'YOU', 'EAT', 'THEM', 'IN', 

In [249]:
indicator = np.zeros((len(words) - 8, 8, len(dictionary)))

for i in range(len(words) - 8):
    eight_integers = text_as_integers[i : i + 8]
    for j in range(8):
        indicator[i, j, eight_integers[j]] = 1

indicator

array([[[0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [250]:
indicator.shape

(172, 8, 26)

In [251]:
indicator.reshape(-1, 26).shape

(1376, 26)

In [415]:
neural_network = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(20, 14, 10, 10, 14, 20), alpha=0.0001, max_iter=10000)

In [416]:
neural_network.fit(indicator.reshape(-1, 26), indicator.reshape(-1, 26))

MLPRegressor(hidden_layer_sizes=(20, 14, 10, 10, 14, 20), max_iter=10000)

In [417]:
input_lines = []
for i in range(0, len(words) - 8, 8):
    input_lines.append(words[i : i + 8])

output_lines = []
for i in range(0, len(words) - 8, 8):
    output_lines.append(dictionary[np.argmax(neural_network.predict(indicator[i]), axis=1)])

for input_line, output_line in zip(input_lines, output_lines):
    differences = 0
    for input_word, output_word in zip(input_line, output_line):
        if input_word != output_word:
            differences += 1
    
    print(f"{' '.join(input_line):40.40s} | {' '.join(output_line):40.40s} | {differences = }")

WOULD YOU LIKE GREEN EGGS AND HAM I      | WOULD LIKE LIKE LIKE A NOT WOULD I       | differences = 5
DO NOT LIKE THEM SAM I AM I              | DO NOT LIKE THEM WOULD I WOULD I         | differences = 2
DO NOT LIKE GREEN EGGS AND HAM WOULD     | DO NOT LIKE LIKE A NOT WOULD WOULD       | differences = 4
YOU LIKE THEM HERE OR THERE I WOULD      | LIKE LIKE THEM HOUSE THEM THERE I WOULD  | differences = 3
NOT LIKE THEM HERE OR THERE I WOULD      | NOT LIKE THEM HOUSE THEM THERE I WOULD   | differences = 2
NOT LIKE THEM ANYWHERE I DO NOT LIKE     | NOT LIKE THEM WOULD I DO NOT LIKE        | differences = 1
GREEN EGGS AND HAM I DO NOT LIKE         | LIKE A NOT WOULD I DO NOT LIKE           | differences = 4
THEM SAM I AM WOULD YOU LIKE THEM        | THEM WOULD I WOULD WOULD LIKE LIKE THEM  | differences = 3
IN A HOUSE WOULD YOU LIKE THEN WITH      | LIKE A HOUSE WOULD LIKE LIKE NOT I       | differences = 4
A MOUSE I DO NOT LIKE THEM IN            | A LIKE I DO NOT LIKE THEM LIKE         

In [171]:
all_sequences_of_8 = np.zeros((len(text_as_integers) - 8, 8), dtype=np.int64)

for i in range(len(text_as_integers) - 8):
    all_sequences_of_8[i] = text_as_integers[i : i + 8]

all_sequences_of_8

array([[24, 25, 15, ...,  2, 10, 13],
       [25, 15,  9, ..., 10, 13,  5],
       [15,  9,  7, ..., 13,  5, 17],
       ...,
       [ 7,  2, 10, ..., 17, 15, 20],
       [ 2, 10, 13, ..., 15, 20, 19],
       [10, 13,  5, ..., 20, 19, 13]])

In [180]:
dictionary[neural_network.predict(all_sequences_of_8[[0]]).astype(np.int64)]

array([['WOULD', 'WOULD', 'LIKE', 'GREEN', 'EAT', 'AM', 'HAM', 'I']],
      dtype='<U8')

In [9]:
array_of_bytes = np.frombuffer(corpus.encode(), np.uint8)
array_of_bytes = array_of_bytes[array_of_bytes <= ord("z")]
array_of_bytes

array([ 84, 104, 101, ..., 115,  46,  10], dtype=uint8)

In [10]:
index_position_to_byte = np.unique(array_of_bytes)
index_position_to_byte

array([ 10,  32,  33,  35,  36,  37,  38,  40,  41,  42,  44,  45,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
        63,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  93,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
       107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
       120, 121, 122], dtype=uint8)

In [11]:
byte_to_index_position = np.zeros(255, np.int32)
byte_to_index_position[index_position_to_byte] = np.arange(81)
byte_to_index_position

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  2,
        0,  3,  4,  5,  6,  0,  7,  8,  9,  0, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25,  0,  0,  0, 26,  0, 27, 28, 29,
       30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
       47, 48, 49, 50, 51, 52, 53,  0, 54,  0,  0,  0, 55, 56, 57, 58, 59,
       60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
       77, 78, 79, 80,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [12]:
def random_segment(array_of_bytes):
    start_index = np.random.randint(0, len(array_of_bytes) - 100)
    stop_index = start_index + 100
    return array_of_bytes[start_index:stop_index]

In [24]:
def to_vector(segment_of_bytes):
    segment_of_bytes = segment_of_bytes
    index_positions = byte_to_index_position[segment_of_bytes]
    
    increasing_integers = np.arange(100)
    
    one_hot = np.zeros((100, 81))
    one_hot[increasing_integers, index_positions] = 1
    return one_hot.reshape(8100)

In [25]:
def from_vector(vectors):
    return index_position_to_byte[np.argmax(vectors.reshape(100, 81), axis=1)].tobytes().decode()

In [26]:
vector = to_vector(random_segment(array_of_bytes))
vector

array([0., 0., 0., ..., 0., 0., 0.])

In [27]:
print(from_vector(vector))

yet methinks, my fathers execution
Was nothing less than bloody tyranny.

MORTIMER.
With silence, ne


In [94]:
neural_network = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(1000,))

In [149]:
many_vectors = np.zeros((1000, 8100))
for i in range(1000):
    many_vectors[i] = to_vector(random_segment(array_of_bytes))

neural_network.partial_fit(many_vectors, many_vectors)

MLPRegressor(hidden_layer_sizes=(1000,))

In [150]:
segment_of_bytes = random_segment(array_of_bytes)
input_text = segment_of_bytes.tobytes().decode()
vector = to_vector(segment_of_bytes)

output_text = from_vector(neural_network.predict(vector.reshape(1, 8100)).reshape(8100))

print(input_text)
print("---------------------------------------------------")
print(output_text)

ath, my lovely Edwards death,
Their kingdoms loss, my woeful banishment,
Should all but answer for t
---------------------------------------------------
                                                                                                    


In [78]:
neural_network.coefs_[0] = np.eye(8100)

In [80]:
neural_network.intercepts_[0] = 0

In [81]:
neural_network.coefs_[0]

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [82]:
neural_network.intercepts_[0]

0

In [ ]:
neural_network = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(1000, 500, 100, 10, 10, 100, 500, 1000))

In [ ]:
segment_of_bytes = random_segment(array_of_bytes)
input_text = segment_of_bytes.tobytes().decode().split("\n")

vectors = to_vectors(segment_of_bytes)
neural_network.partial_fit(vectors, vectors)

output_text = from_vectors(neural_network.predict(vectors)).split("\n")

while len(output_text) < len(input_text):
    output_text.append("")
for input_line, output_line in zip(input_text, output_text):
    print(f"{input_line:50.50s} | {output_line:50.50s}")

In [ ]:
first_half_network = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(1000, 500, 100, 10))
first_half_network.__sklearn_is_fitted__ = lambda: True
first_half_network.n_layers_ = 5
first_half_network.coefs_ = neural_network.coefs_[:4]
first_half_network.intercepts_ = neural_network.intercepts_[:4]
first_half_network.out_activation_ = "relu"

In [ ]:
second_half_network = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(10, 100, 500, 1000))
second_half_network.__sklearn_is_fitted__ = lambda: True
second_half_network.n_layers_ = 6
second_half_network.coefs_ = neural_network.coefs_[4:]
second_half_network.intercepts_ = neural_network.intercepts_[4:]
second_half_network.out_activation_ = "identity"

In [ ]:
encoded = first_half_network.predict(vectors)
encoded.shape

In [ ]:
decoded = second_half_network.predict(encoded)

print(from_vectors(decoded))